In [1]:
import pandas as pd
from pandas_datareader import data as pdr

import yfinance as yf
yf.pdr_override()

In [2]:
dividend_aristocrats = ['MMM', 'SWK', 'SYY', 'TGT']
Safe_bets = ['AAPL', 'GOOG', 'AMZN']

In [3]:
open_positions = ['SCHB', 'SCHD', 'VTI', 'INTC']

In [4]:
data = pdr.get_data_yahoo('VTI', start='2016-01-01', end='2023-03-02')

[*********************100%***********************]  1 of 1 completed


In [5]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2016-01-04,102.580002,102.949997,101.580002,102.739998,90.817894,6138300
2016-01-05,102.900002,103.180000,102.300003,102.970001,91.021202,3652300
2016-01-06,101.529999,102.230003,100.949997,101.589996,89.801338,3732800
2016-01-07,99.900002,100.800003,98.870003,99.099998,87.600288,9256200
2016-01-08,99.610001,99.989998,97.779999,97.970001,86.601410,7015800
...,...,...,...,...,...,...
2023-02-23,202.089996,202.460007,199.380005,201.729996,201.729996,3440100
2023-02-24,199.289993,199.929993,198.160004,199.479996,199.479996,3020600
2023-02-27,201.309998,202.029999,199.720001,200.190002,200.190002,2850300


In [6]:
#Variables/constants for EMA Calculation:
NUM_PERIODS_FAST = 10 # Static time period parameter for the fast EMA
K_FAST = 2 / (NUM_PERIODS_FAST + 1) # Static smoothing factor parameter for fast EMA
ema_fast = 0
ema_fast_values = [] #we will hold fast EMA values for visualization purposes

In [7]:
NUM_PERIODS_SLOW = 40 #Static time period parameter for slow EMA
K_SLOW = 2 / (NUM_PERIODS_SLOW + 1) #Static smoothing factor parameter for slow EMA
ema_slow = 0
ema_slow_values = [] #we will hold slow EMA values for visualization purposes

In [8]:
apo_values = [] #track computed absolute price oscillator value signals

In [9]:
last_buy_price = 0 #Price at which last buy trade was made, used to prevent over-trading at/around the same price
last_sell_price = 0 #Price at which last sell trade was made, used to prevent over-trading at/around the same price
position = 0 #Current position of the trading strategy
buy_sum_price_qty = 0 #Summation of products of buy_trade_price and buy_trade_qty for every buy Trade made since last time being flat
buy_sum_qty = 0 #Summation of buy_trade_qty for every buy Trade made since last time being flat
sell_sum_price_qty = 0 #Summation of products of sell_trade_price and sell_trade_qty for every sell Trade made since last time being flat
sell_sum_qty = 0 #Summation of sell_trade_qty for every sell Trade made since last time being flat
open_pnl = 0 #Open/Unrealized PnL marked to market
closed_pnl = 0 #Closed/Realized PnL so far

In [10]:
#Constants that define strategy behavior/thresholds
APO_VALUE_FOR_BUY_ENTRY = -10 # APO trading signal value below which to enter buy-orders/long-position
APO_VALUE_FOR_SELL_ENTRY = 10 # APO trading signal value above which to enter sell-orders/short-position
MIN_PRICE_MOVE_FROM_LAST_TRADE = 10 # Minimum price change since last trade before considering trading again, this is to prevent over-trading at/around same prices
NUM_SHARES_PER_TRADE = 10 # Number of shares to buy/sell on every trade
MIN_PROFIT_TO_CLOSE = 10*NUM_SHARES_PER_TRADE # Minimum Open/Unrealized profit at which to close positions and lock profits

In [11]:
import statistics as stats
import math as math

In [12]:
# Constants/variables that are used to compute standard deviation as a volatility measure
SMA_NUM_PERIODS = 20 # look back period
price_history = [] # history of prices

In [13]:
close = data['Close']
for close_price in close:
    price_history.append(close_price)
    if len(price_history) > SMA_NUM_PERIODS:
        del(price_history[0])
    
    sma = stats.mean(price_history)
    variance = 0
    for hist_price in price_history:
        variance = variance + ((hist_price - sma) ** 2)
        
    stdev = math.sqrt(variance / len(price_history))
    stdev_factor = stdev/15
    if stdev_factor == 0:
        stdev_factor = 1
        
    # This section updates fast and slow EMA and computes APO trading signal
    if (ema_fast == 0): # first observation
        ema_fast = close_price
        ema_slow = close_price
    else:
        ema_fast = (close_price - ema_fast) * K_FAST*stdev_factor + ema_fast
        ema_slow = (close_price - ema_slow) * K_SLOW*stdev_factor + ema_slow

    ema_fast_values.append(ema_fast)
    ema_slow_values.append(ema_slow)

    apo = ema_fast - ema_slow
    apo_values.append(apo)

In [14]:
# This section prepares the dataframe from the trading strategy results and visualizes the results
data = data.assign(ClosePrice=pd.Series(close, index=data.index))
data = data.assign(Fast10DayEMA=pd.Series(ema_fast_values, index=data.index))
data = data.assign(Slow40DayEMA=pd.Series(ema_slow_values, index=data.index))
data = data.assign(APO=pd.Series(apo_values, index=data.index))

In [15]:
data

,Open,High,Low,Close,Adj Close,Volume,ClosePrice,Fast10DayEMA,Slow40DayEMA,APO
Date,,,,,,,,,,
2016-01-04,102.580002,102.949997,101.580002,102.739998,90.817894,6138300,102.739998,102.739998,102.739998,0.000000
2016-01-05,102.900002,103.180000,102.300003,102.970001,91.021202,3652300,102.970001,102.740318,102.740084,0.000235
2016-01-06,101.529999,102.230003,100.949997,101.589996,89.801338,3732800,101.589996,102.731901,102.737826,-0.005925
2016-01-07,99.900002,100.800003,98.870003,99.099998,87.600288,9256200,99.099998,102.664320,102.719665,-0.055345
2016-01-08,99.610001,99.989998,97.779999,97.970001,86.601410,7015800,97.970001,102.550609,102.688797,-0.138189
...,...,...,...,...,...,...,...,...,...,...
2023-02-23,202.089996,202.460007,199.380005,201.729996,201.729996,3440100,201.729996,201.770269,198.334384,3.435885
2023-02-24,199.289993,199.929993,198.160004,199.479996,199.479996,3020600,199.479996,201.688369,198.345375,3.342994
2023-02-27,201.309998,202.029999,199.720001,200.190002,200.190002,2850300,200.190002,201.631455,198.364173,3.267282


In [16]:
if data.iloc[-1, 9] < -10:
    print('The APO is at', data.iloc[-1, 9], 'which indicates a buy signal')
elif data.iloc[-1, 9] > -10 and data.iloc[-1, 9] < 10:
    print('The APO is at', data.iloc[-1, 9], 'which indicates a hold signal unless the price is')
else:
    print('The APO is at', data.iloc[-1, 9], 'which indicates a sell signal')

The APO is at 3.0894849686706607 which indicates a hold signal unless the price is
